In [1]:
import pandas as pd
from math import sqrt
from math import pi
from math import exp

In [2]:
naive_bayes_db = pd.read_csv('df_with_transformed_time.csv', index_col=False)

In [3]:
def separate_by_color(dataframe):
    dict_od_separated_dataframes = dict()
    for i in dataframe['landing_safety'].unique():
        df_separated = dataframe.loc[dataframe['landing_safety'] == i]
        dict_od_separated_dataframes[i] = df_separated
    return dict_od_separated_dataframes

In [4]:
def summarize_dataframe(dataframe):
    summaries = [dataframe.loc[:, dataframe.columns != 'landing_safety'].mean(), dataframe.loc[:, dataframe.columns != 'landing_safety'].std(),len(dataframe.loc[:, dataframe.columns != 'landing_safety'])]
    return summaries

In [5]:
def summarize_by_class(dataframe):
    separated = separate_by_color(dataframe)
    summaries = dict()
    for class_value, rows in separated.items():
        summaries[class_value] = summarize_dataframe(rows)
    return summaries

In [6]:
def calculate_probability(x, mean, stdev):
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

In [7]:
def calculate_class_probabilities(summaries, row):
    total_rows = len(test)
    probabilities = dict()
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = summaries[class_value][-1]/float(total_rows)
        for i in range(len(class_summaries[0])):
            mean = class_summaries[0][i]
            stdev = class_summaries[1][i]
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
    return probabilities

In [8]:
def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label

In [9]:
def naive_bayes(train, test):
    summarize = summarize_by_class(train)
    predictions = list()
    for index, row in test.iterrows():
        output = predict(summarize, row[1:11].tolist())
        predictions.append(output)
    return(predictions)

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(naive_bayes_db.sample(500), test_size=0.3, random_state=1)

In [11]:
predicted_values = naive_bayes(train, test)

In [12]:
d = list()
for a in range(len(test['landing_safety'])-1):
    d.append((test['landing_safety'].values[a],predicted_values[a]))

In [13]:
for a in d:
    print(a)

('green', 'green')
('green', 'green')
('green', 'green')
('green', 'green')
('red', 'green')
('red', 'green')
('green', 'green')
('green', 'green')
('red', 'red')
('green', 'green')
('green', 'green')
('red', 'green')
('red', 'green')
('yellow', 'green')
('green', 'green')
('red', 'green')
('green', 'green')
('green', 'green')
('green', 'green')
('green', 'green')
('yellow', 'green')
('green', 'green')
('green', 'green')
('green', 'green')
('red', 'red')
('yellow', 'green')
('green', 'green')
('red', 'green')
('green', 'green')
('red', 'green')
('red', 'green')
('red', 'red')
('red', 'green')
('green', 'green')
('green', 'green')
('green', 'green')
('red', 'green')
('green', 'green')
('red', 'green')
('green', 'yellow')
('green', 'green')
('green', 'green')
('red', 'green')
('green', 'green')
('red', 'green')
('green', 'red')
('green', 'green')
('green', 'green')
('green', 'green')
('green', 'green')
('green', 'green')
('green', 'green')
('green', 'green')
('yellow', 'green')
('green',

In [14]:
from sklearn import metrics

In [15]:
print(metrics.classification_report(test['landing_safety'].values, predicted_values))

              precision    recall  f1-score   support

       green       0.69      0.98      0.81       102
         red       0.60      0.09      0.16        32
      yellow       0.00      0.00      0.00        16

    accuracy                           0.69       150
   macro avg       0.43      0.36      0.33       150
weighted avg       0.60      0.69      0.59       150

